<a href="https://colab.research.google.com/github/maoo1/meme-propaganda-detector/blob/master/MEME_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[sem eval task](https://propaganda.math.unipd.it/semeval2024task4/teampage.php?passcode=7a4c50dc60f44593a07529d2253593e9)

to do:
- test & finish training loop
- clean up & visualize

In [1]:
#!git pull origin master


# Setting up libraries and mounting google drive




In [2]:
# install dependencies and codebase
# transformers Hugging Face library: access to BERT / GPT
!pip install torch transformers datasets tqdm gdown==v4.6.3
#!pip install evaluate
#!pip install sklearn_hierarchical_classification
!mkdir checkpoints

mkdir: cannot create directory ‘checkpoints’: File exists


In [3]:
# mounting data and files to drive
# hypothetically can upload SemEval datasets to drive for easier access + we would both have access
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# HIERARCHY

# Preprocessing the Data

In [4]:
import json
from dataclasses import dataclass
from typing import List, Dict, Tuple, Union

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

import numpy as np

creating Meme object and dataset

In [5]:
# represents single meme as text, label
@dataclass
class Meme:
  text: str
  labels: Union[List[str], None] # labels can be none for test data

  @staticmethod
  def from_dict(data: dict):
    text = data["text"]
    labels = data.get("labels")
    if len(labels) == 0:
        labels = ['None']
    return Meme(text=text, labels=labels)

# custom dataset for meme data, supporting multi-label classification
class MemeDataset(Dataset):

  tokenizer: AutoTokenizer.from_pretrained("roberta-base")

  def __init__(self, tokenizer, data: List[Dict], label_encoder):
    MemeDataset.tokenizer = tokenizer
    self.examples = [Meme.from_dict(item) for item in data]
    self.label_encoder = label_encoder

    if label_encoder:
      # encode labels if available
      self.encoded_labels = [label_encoder.fit_transform([item.labels]) for item in self.examples]
    else:
      self.encoded_labels = None

  def __len__(self):
    return len(self.examples)

  def __iter__(self):
    return iter(self.examples)

  def __getitem__(self, idx):
    example = self.examples[idx]
    encoding = self.tokenizer(
        example.text,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    labels = self.encoded_labels[idx] if self.encoded_labels else None
    return {"text": example.text, "labels": labels}

  # batch processing
  @staticmethod
  def collate_fn(batched_samples: List[Meme]):

    batched_texts = [sample['text'] for sample in batched_samples]
    batched_labels = [sample['labels'] for sample in batched_samples]# if sample.labels is not None]

    text_encoding = MemeDataset.tokenizer(
        batched_texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    text_encoding['labels'] = torch.tensor(np.squeeze(np.array(batched_labels)), dtype=torch.float)

    return text_encoding

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
from sklearn.preprocessing import MultiLabelBinarizer
class_labels = ['Appeal to authority', 'Appeal to fear/prejudice', 'Bandwagon',
                'Black-and-white Fallacy/Dictatorship', 'Causal Oversimplification',
                'Doubt', 'Exaggeration/Minimisation', 'Flag-waving',
                'Glittering generalities (Virtue)', 'Loaded Language',
                "Misrepresentation of Someone's Position (Straw Man)",
                'Name calling/Labeling', 'Obfuscation, Intentional vagueness, Confusion',
                'Presenting Irrelevant Data (Red Herring)', 'Reductio ad hitlerum',
                'Repetition', 'Slogans', 'Smears', 'Thought-terminating cliché',
                'Whataboutism', 'None']
mlb = MultiLabelBinarizer(classes=class_labels)

In [15]:
def load_json(file_path):
  with open(file_path, 'r') as f:
    return json.load(f)

# initializing datasets
def initialize_datasets(tokenizer, dataset_paths: Dict, label_encoder) -> dict:

  raw_datasets = {}

  # loading raw datasets from file paths
  for split_name, file_path in dataset_paths.items():
    raw_datasets[split_name] = load_json(file_path)

  # convert raw datasets to MemeDataset objects
  split_datasets = {}
  for split_name, split_data in raw_datasets.items():
    split_datasets[split_name] = MemeDataset(tokenizer, list(split_data), label_encoder)

  return split_datasets

dataset_paths = {
  "train": '/content/drive/MyDrive/NLP_FINAL/train.json',
  "validation": '/content/drive/MyDrive/NLP_FINAL/validation.json',
  # "dev_unlabeled":  '/content/drive/MyDrive/NLP_FINAL/dev_unlabeled.json',
  "test":  '/content/drive/MyDrive/NLP_FINAL/dev_subtask1_en.json'
}

Loading the data

In [8]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

"""
datasets = {}
for split_name in raw_datasets.keys():
  split_data = list(raw_datasets[split_name])

  datasets[split_name] = MemeDataset(tokenizer, split_data)
"""
# loading datasets
datasets = initialize_datasets(tokenizer, dataset_paths, mlb)

length_train = len(datasets['train'])
length_val = len(datasets['validation'])
# length_dev_unlabaled = len(datasets['dev_unlabeled'])
length_test = len(datasets['test'])

print("num training examples:", length_train)
print("num validation examples:", length_val)
print("num test examples:", length_test)

#validation_dataloader = DataLoader(datasets['validation'],
#                                   batch_size=64,
#                                   shuffle=False,
#                                   collate_fn=MemeDataset.collate_fn,
#                                   num_workers=2)

for i in range(0, 6):
    print(datasets['train'][i]['text'])
    print(datasets['train'][i]['labels'])
    print(mlb.inverse_transform(datasets['train'][i]['labels']))

num training examples: 7000
num validation examples: 500
num test examples: 1000
THIS IS WHY YOU NEED\n\nA SHARPIE WITH YOU AT ALL TIMES
[[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[('Black-and-white Fallacy/Dictatorship',)]
GOOD NEWS!\n\nNAZANIN ZAGHARI-RATCLIFFE AND ANOOSHEH ASHOORI HAVE BEEN RELEASED\n\nAfter years of being unjustly detained in Iran, they are making their way safely back to the UK.
[[0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]]
[('Glittering generalities (Virtue)', 'Loaded Language')]
PAING PHYO MIN IS FREE!
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
[('None',)]
Move your ships away!\n\noooook\n\nMove your ships away!\n\nNo, and I just added 10 more
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
[('None',)]
WHEN YOU'RE THE FBI, THEY LET YOU DO IT.
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]]
[('Thought-terminating cliché',)]
PUTIN'S SECRET CAMOUFLAGE ARMY
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
[('None',)]


# Checking if Preprocessing Data Correct

In [9]:
def check_label_encoding(dataset, mlb):
    """
    Checks if all items in the dataset have been properly encoded by MLB.
    """
    for i, item in enumerate(dataset.examples):
        if item.labels is not None:
            # Encode labels
            encoded = mlb.transform([item.labels])[0]
            # Convert to 2D numpy array for inverse_transform
            decoded = mlb.inverse_transform(np.array([encoded]))[0]

            # Check that original and decoded labels match
            assert set(item.labels) == set(decoded), \
                f"Mismatch in encoding for sample {i}.\nOriginal: {item.labels}\nDecoded: {decoded}"

    print(f"All labels in {len(dataset.examples)} examples are properly encoded!")

check_label_encoding(datasets['train'], mlb)
check_label_encoding(datasets['validation'], mlb)
check_label_encoding(datasets['test'], mlb)

All labels in 7000 examples are properly encoded!
All labels in 500 examples are properly encoded!
All labels in 1000 examples are properly encoded!


# TRAINING

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import Optimizer, AdamW
from tqdm.notebook import tqdm
from transformers import AutoModelForSequenceClassification#, Trainer, TrainingArguments
from sklearn.metrics import classification_report
from torch.nn import BCEWithLogitsLoss

In [11]:
def evaluate(model: nn.Module, dataloader: DataLoader) -> float:
    model.eval()

    all_predictions = []
    all_labels = []
    with tqdm(dataloader, desc=f"Eval", total=len(dataloader)) as tq:
        for batch in tq:
            with torch.no_grad():
                text = batch['input_ids'].to(model.device)
                attention = batch['attention_mask'].to(model.device)
                label_encoding = batch['labels'].to(model.device)

                logits = model(input_ids=text, attention_mask=attention, labels=label_encoding).logits # Hint: logit should be of size (batch_size, 2)

                predictions = (logits > 0.5).int().tolist()
                labels = label_encoding.tolist() # Hint: should be a list [0, 1, ...] of ground-truth labels

                all_predictions += predictions
                all_labels += labels

    # compute f1 score
    f1 = classification_report(y_true=all_predictions, y_pred=all_labels, target_names=class_labels, output_dict=True)['micro avg']['f1-score']

    print(f"F1: {f1}")
    return f1

def train_one_epoch(model: nn.Module, dataloader: DataLoader, optimizer: Optimizer, epoch: int, criterion):
    model.train()

    with tqdm(dataloader, desc=f"Train Ep {epoch}", total=len(dataloader)) as tq:
        for batch in tq:
            text = batch['input_ids'].to(model.device)
            attention = batch['attention_mask'].to(model.device)
            label_encoding = batch['labels'].to(model.device)

            optimizer.zero_grad()

            result = model(input_ids=text, attention_mask=attention)
            loss = criterion(result.logits, label_encoding)

            loss.backward()
            optimizer.step()

            tq.set_postfix({"loss": loss.detach().item()}) # for printing better-looking progress bar

In [12]:
def main():
    # hyper-parameters
    batch_size = 32
    learning_rate = 5e-5
    num_epochs = 10
    model_name = "roberta-base"

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(class_labels))

    # load model on GPU.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)

    criterion = BCEWithLogitsLoss()

    datasets = initialize_datasets(tokenizer, dataset_paths, mlb)

    train_dataloader = DataLoader(datasets['train'],
                                  batch_size=batch_size,
                                  shuffle=True,
                                  collate_fn=MemeDataset.collate_fn,
                                  num_workers=2)

    validation_dataloader = DataLoader(datasets['validation'],
                                      batch_size=batch_size,
                                      shuffle=False,
                                      collate_fn=MemeDataset.collate_fn,
                                      num_workers=2)

    # training loop
    best_f1 = 0.0
    for epoch in range(1, num_epochs + 1):
        train_one_epoch(model, train_dataloader, optimizer, epoch, criterion)
        valid_f1 = evaluate(model, validation_dataloader)

        if valid_f1 > best_f1:
            model.save_pretrained('./checkpoints')
            best_f1 = valid_f1

In [13]:
main()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train Ep 1:   0%|          | 0/219 [00:00<?, ?it/s]

Eval:   0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


F1: 0.07223942208462332


...

F1: 0.4667535853976532


Train Ep 10:   0%|          | 0/219 [00:00<?, ?it/s]

Eval:   0%|          | 0/16 [00:00<?, ?it/s]

F1: 0.45


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# TODO: Load your best trained model from `./checkpoints/` and report the test set accuracy.
model = AutoModelForSequenceClassification.from_pretrained('./checkpoints')

datasets = initialize_datasets(tokenizer, dataset_paths, mlb)
# TODO: Load the test dataset
test_dataloader = DataLoader(datasets['test'],
                            batch_size=64,
                            shuffle=False,
                            collate_fn=MemeDataset.collate_fn,
                            num_workers=2)

# TODO: evaluate the model on the test set
evaluate(model, test_dataloader)

Eval:   0%|          | 0/16 [00:00<?, ?it/s]

F1: 0.45423518461061124


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.45423518461061124